In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use strict;
use warnings;
use Math::Complex;
use List::Util qw(sum);
use sml; # Statistical Machine Learning Library
use Chart::Plotly::Plot;
use Chart::Plotly::Trace::Scatter;
IPerl->load_plugin('Chart::Plotly');

# k-Nearest Neighbors

A simple but powerful approach for making predictions is to use the most similar historical
examples to the new data. This is the principle behind the k-Nearest Neighbors algorithm. In
this tutorial you will discover how to implement the k-Nearest Neighbors algorithm from scratch
with Python. After completing this tutorial you will know:
 How to calculate the similarity between two pieces of data.
 How to make a prediction using the most similar historical records.
 How to use k-Nearest Neighbors for both classification and regression problems.
Let’s get started.


# 13.1 Description

This section will provide a brief background on the k-Nearest Neighbors algorithm that we will
implement in this tutorial and the Abalone dataset to which we will apply it.

# 13.1.1 k-Nearest Neighbors

The k-Nearest Neighbors algorithm or KNN for short is a very simple technique. The entire
training dataset is stored. When a prediction is required, the k-most similar records to a
new record from the training dataset are then located. From these neighbors, a summarized
prediction is made. Similarity between records can be measured many different ways. A problem
or data-specific method can be used. Generally, with tabular data, a good starting point is the
Euclidean distance.
Once the neighbors are discovered, the summary prediction can be made by returning the
most common outcome or taking the average. As such, KNN can be used for classification
or regression problems. There is no model to speak of other than holding the entire training
dataset. Because no work is done until a prediction is required, KNN is often referred to as a
lazy learning method.


# 13.1.2 Abalone Dataset

In this tutorial we will use the Abalone Dataset. This dataset involves the prediction of the age
of abalone. The baseline performance on the problem is approximately 16% or an RMSE of
approximately 3.2 rings. You can learn more about it in Appendix A, Section A.8. Download
the dataset and save it into your current working directory with the filename abalone.csv.


# 13.2 Tutorial

This tutorial is broken down into 5 parts:
1. Euclidean Distance.
2. Get Neighbors.
3. Make Predictions.
4. Abalone Case Study as Classification.
5. Abalone Case Study as Regression.
These steps will teach you the fundamentals of implementing and applying the k-Nearest
Neighbors algorithm for classification and regression predictive modeling problems.


# 13.2.1 Euclidean Distance



The first step needed is to calculate the distance between two rows in a dataset. Rows of data are mostly made up of numbers, and an easy way to calculate the distance between two rows or vectors of numbers is to draw a straight line. This makes sense in 2D or 3D and scales nicely to higher dimensions. We can calculate the straight line distance between two vectors using the Euclidean distance measure. It is calculated as the square root of the sum of the squared differences between the two vectors:


\text{{distance}} = \sqrt{\sum_{i=1}^{n}(x_{1i} - x_{2i})^2} \quad (13.1)


Where \(x_1\) is the first row of data, \(x_2\) is the second row of data, and \(i\) is the index to a specific column as we sum across all columns. With Euclidean distance, the smaller the value, the more similar two records will be. A value of 0 means that there is no difference between two records. Below is a function named \texttt{euclidean\_distance()} that implements this in Python.







In [2]:
my $euclidean_distance = sub{
    my ($self, $row1, $row2) = @_;
    my $distance = 0.0;
    for my $i (0 .. $#{$row1} - 1) {
        $distance += ($row1->[$i] - $row2->[$i])**2;
    }
    return sqrt($distance);
};
sml->add_to_class('sml','euclidean_distance',$euclidean_distance);

*sml::euclidean_distance

You can see that the function assumes that the last column in each row is an output value
which is ignored from the distance calculation. We can test this distance function with a small
contrived classification dataset. We will use this dataset a few times as we construct the elements
needed for the KNN algorithm

Below is a plot of the dataset using different colors to show the different classes for each
point.

In [3]:
# Datos del conjunto de datos
my ($X1, $X2, $Y) = (
    [2.7810836, 1.465489372, 3.396561688, 1.38807019, 3.06407232, 7.627531214, 5.332441248, 6.922596716, 8.675418651, 7.673756466],
    [2.550537003, 2.362125076, 4.400293529, 1.850220317, 3.005305973, 2.759262235, 2.088626775, 1.77106367, -0.242068655, 3.508563011],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
);

# Colores y símbolos para las clases
my @colors = map { $_ == 0 ? 'red' : 'blue' } @$Y;
my @symbols = map { $_ == 0 ? 'diamond' : 'square' } @$Y;

# Tamaño del símbolo (ajústalo según sea necesario)
my $symbol_size = 10;

# Crear el rastro (trace) Scatter
my $scatter_trace = Chart::Plotly::Trace::Scatter->new(
    x      => $X1,
    y      => $X2,
    mode   => 'markers',
    marker => { color => \@colors, size => $symbol_size, symbol => \@symbols }
);

# Crear el gráfico
my $plot = Chart::Plotly::Plot->new(
    traces => [$scatter_trace],
    layout => { title => { text => 'Conjunto de Datos para Prueba de Regresión Logística' } }
);




Putting this all together, we can write a small example to test our distance function by
printing the distance between the first row and all other rows. We would expect the distance
between the first row and itself to be 0, a good thing to look out for. The full example is listed
below.

In [4]:
# Conjunto de datos
my $dataset = [
    [2.7810836, 2.550537003, 0],
    [1.465489372, 2.362125076, 0],
    [3.396561688, 4.400293529, 0],
    [1.38807019, 1.850220317, 0],
    [3.06407232, 3.005305973, 0],
    [7.627531214, 2.759262235, 1],
    [5.332441248, 2.088626775, 1],
    [6.922596716, 1.77106367, 1],
    [8.675418651, -0.242068655, 1],
    [7.673756466, 3.508563011, 1]
];

# Seleccionar la primera fila como referencia
my $row0 = $dataset->[0];

# Calcular la distancia para cada fila en el conjunto de datos
for my $row (@$dataset) {
    my $distance = sml->euclidean_distance($row0, $row);
    print("$distance\n");
}


0
1.32901739152758
1.94946466556532
1.55914393855405
0.535628072193849
4.85094018698641
2.59283375995051
4.21422704263287
6.52240998822834
4.98558538244979


Running this example prints the distances between the first row and every row in the dataset,
including itself.

Now it is time to use the distance calculation to locate neighbors within a dataset.

# 13.2.2 Get Neighbors


Neighbors for a new piece of data in the dataset are the k closest instances, as defined by our
distance measure. To locate the neighbors for a new piece of data within a dataset we must
first calculate the distance between each record in the dataset to the new piece of data. We can
do this using our distance function above. Once distances are calculated, we must sort all of the
records in the training dataset by their distance to the new data. We can then select the top k
to return as the most similar neighbors.
We can do this by keeping track of the distance for each record in the dataset as a tuple,
sort the list of tuples by the distance (in descending order) and then retrieve the neighbors.
Below is a function named get neighbors() that implements this.


In [5]:
# Locate the most similar neighbors
my $get_neighbors = sub{
    my ($self, $train, $test_row, $num_neighbors) = @_;
    my @distances;
    for my $train_row (@$train) {
        my $dist = sml->euclidean_distance($test_row, $train_row);
        push @distances, [$train_row, $dist];
    }
    @distances = sort { $a->[1] <=> $b->[1] } @distances;
    my @neighbors;
    for my $i (0 .. $num_neighbors - 1) {
        push @neighbors, $distances[$i][0];
    }
    return \@neighbors;
};
sml->add_to_class('sml','get_neighbors',$get_neighbors);

*sml::get_neighbors

You can see that the euclidean distance() function developed in the previous step is used
to calculate the distance between each train row and the new test row. The list of train row
and distance tuples is sorted where a custom key is used ensuring that the second item in the
tuple (tup[1]) is used in the sorting operation.
Finally, a list of the num neighbors most similar neighbors to test row is returned. We
can test this function with the small contrived dataset prepared in the previous section. The
complete example is listed below.


In [6]:

# Conjunto de datos
my $dataset = [
    [2.7810836, 2.550537003, 0],
    [1.465489372, 2.362125076, 0],
    [3.396561688, 4.400293529, 0],
    [1.38807019, 1.850220317, 0],
    [3.06407232, 3.005305973, 0],
    [7.627531214, 2.759262235, 1],
    [5.332441248, 2.088626775, 1],
    [6.922596716, 1.77106367, 1],
    [8.675418651, -0.242068655, 1],
    [7.673756466, 3.508563011, 1]
];

# Obtener vecinos para la primera instancia del conjunto de datos
my $neighbors = sml->get_neighbors($dataset, $dataset->[0], 3);

# Imprimir los vecinos
for my $neighbor (@$neighbors) {
    print("@$neighbor\n");
}


2.7810836 2.550537003 0
3.06407232 3.005305973 0
1.465489372 2.362125076 0


Running this example prints the 3 most similar records in the dataset to the first record, in
order of similarity. As expected, the first record is the most similar to itself and is at the top of
the list.

Now that we know how to get neighbors from the dataset, we can use them to make
predictions

### 13.2.3 Make Predictions

The most similar neighbors collected from the training dataset can be used to make predictions.
In the case of classification, we can return the most represented class among the neighbors.
We can achieve this by performing the max() function on the list of output values from the
neighbors. Given a list of class values observed in the neighbors, the max() function takes a set
of unique class values and calls the count on the list of class values for each class value in the
set. Below is the function named predict classification() that implements this.

In [7]:
# Subrutina para predecir la clasificación con vecinos
my $predict_classification = sub{
    my ($self, $train, $test_row, $num_neighbors) = @_;
    my $neighbors = get_neighbors($train, $test_row, $num_neighbors);
    my @output_values = map { $_->[-1] } @$neighbors;
    my $prediction = max(keys %{{ map { $_ => 1 } @output_values }});
    return $prediction;
};
sml->add_to_class('sml','predict_classification',$predict_classification);

*sml::predict_classification

We can test this function on the above contrived dataset. Below is a complete example.


In [8]:

# Conjunto de datos
my $dataset = [
    [2.7810836, 2.550537003, 0],
    [1.465489372, 2.362125076, 0],
    [3.396561688, 4.400293529, 0],
    [1.38807019, 1.850220317, 0],
    [3.06407232, 3.005305973, 0],
    [7.627531214, 2.759262235, 1],
    [5.332441248, 2.088626775, 1],
    [6.922596716, 1.77106367, 1],
    [8.675418651, -0.242068655, 1],
    [7.673756466, 3.508563011, 1]
];

# Hacer una predicción de clasificación para la primera instancia del conjunto de datos
my $prediction = sml->predict_classification($dataset, $dataset->[0], 3);

# Imprimir el resultado
print("Expected $dataset->[0][-1], Got $prediction.\n");


Error: Undefined subroutine &main::get_neighbors called at reply input line 4.


Running this example prints the expected classification of 0 and the actual classification
predicted from the 3 most similar neighbors in the dataset.

We can imagine how the predict classification() function can be changed to calculate
the mean value of the outcome values. Below is a quick example named predict regression()
that we will use later.

In [9]:
my $predict_regression = sub{
    my ($self, $train, $test_row, $num_neighbors) = @_;
    my $neighbors = sml->get_neighbors($train, $test_row, $num_neighbors);
    my @output_values = map $_->[-1], @$neighbors;
    my $prediction = sum(@output_values) / @$neighbors;
    return $prediction;
};
sml->add_to_class('sml','predict_regression',$predict_regression);

*sml::predict_regression

We now have all of the pieces to make predictions with KNN. Let’s apply it to the Abalone
dataset.

# 13.2.4 Abalone Case Study as Classification


In this section we will apply the k-Nearest Neighbors algorithm to the Abalone dataset. The
first step is to load the dataset and convert the loaded data to numbers that we can use with
the Euclidean distance calculation. For this we will use the helper function load csv() to load
the file, str column to float() to convert string numbers to floats and str column to int()
to convert the sex column (column 0) to integer values.
We will evaluate the algorithm using k-fold cross-validation with 5 folds. This means that
4177
5 = 835.4 or just over 830 records will be in each fold. We will use the helper functions evaluate algorithm() to evaluate the algorithm with cross-validation and accuracy metric()
to calculate the accuracy of predictions. The complete example is listed below.

In [10]:

# Split a dataset into k folds
#igual
my $cross_validation_split = sub{
    my ($self, $dataset, $n_folds) = @_;
    my @dataset_split;
    my @dataset_copy = @$dataset;
    my $fold_size = int(@$dataset / $n_folds);
    for (1 .. $n_folds) {
        my @fold;
        while (@fold < $fold_size) {
            my $index = int(rand(@dataset_copy));
            push @fold, splice @dataset_copy, $index, 1;
        }
        push @dataset_split, \@fold;
    }
    return \@dataset_split;
};

sml->add_to_class('sml','cross_validation_split',$cross_validation_split);

# Calculate accuracy percentage
#No igual
my $accuracy_metric = sub{
    my ($self, $actual, $predicted) = @_;
    my $correct = 0;
    for my $i (0 .. @$actual - 1) {
        $correct++ if $actual->[$i] == $predicted->[$i];
    }
    return $correct / @$actual * 100.0;
};
sml->add_to_class('sml','accuracy_metric',$accuracy_metric);


# Evaluate an algorithm using a cross-validation split
#No igual
my $evaluate_algorithm = sub{
    my ($self, $dataset, $algorithm, $n_folds, @args) = @_;
    my $folds = sml->cross_validation_split($dataset, $n_folds);
    my @scores;
    for my $fold (@$folds) {
        my @train_set = @$folds;
        @train_set = grep { $_ ne $fold } @train_set;
        @train_set = map @$_, @train_set;
        my @test_set;
        for my $row (@$fold) {
            my @row_copy = @$row;
            push @test_set, \@row_copy;
            $row_copy[-1] = undef;
        }
        my $predicted = $algorithm->(\@train_set, \@test_set, @args);
        my @actual = map $_->[-1], @$fold;
        my $accuracy = sml->accuracy_metric(\@actual, $predicted);
        push @scores, $accuracy;
    }
    return \@scores;
};
sml->add_to_class('sml','evaluate_algorithm',$evaluate_algorithm);

#No igual
my $predict_classification = sub{
    my ($self, $train, $test_row, $num_neighbors) = @_;
    my $neighbors = sml->get_neighbors($train, $test_row, $num_neighbors);
    my @output_values = map $_->[-1], @$neighbors;
    my %counts;
    $counts{$_}++ for @output_values;
    my $prediction = (sort { $counts{$b} <=> $counts{$a} } keys %counts)[0];
    return $prediction;
};
sml->add_to_class('sml','predict_classification',$predict_classification);

# kNN Algorithm
#igual
sub k_nearest_neighbors {
    my ($train, $test, $num_neighbors) = @_;
    my @predictions;
    for my $row (@$test) {
        my $output = sml->predict_classification($train, $row, $num_neighbors);
        push @predictions, $output;
    }
    return \@predictions;
}

# Test the kNN on the Abalone dataset
srand(1);

# load and prepare data
my $filename = 'abalone.csv';
my $dataset = sml->load_csv($filename);
for my $i (1 .. $#{$dataset->[0]}) {
    sml->str_column_to_float($dataset, $i);
}

# convert the first column to integers
my $lookup = sml->str_column_to_int($dataset, 0);

# evaluate algorithm
my $n_folds = 5;
my $num_neighbors = 5;
my $scores = sml->evaluate_algorithm($dataset, \&k_nearest_neighbors, $n_folds, $num_neighbors);
print 'Scores: ' . join(', ', @$scores) . "\n";
printf 'Mean Accuracy: %.3f%%\n', sum(@$scores) / @$scores;


Scores: 22.874251497006, 22.7544910179641, 26.2275449101796, 25.1497005988024, 22.9940119760479
Mean Accuracy: 24.000%\n

1

Warning: Subroutine sml::cross_validation_split redefined at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/sml.pm line 19.

Subroutine sml::accuracy_metric redefined at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/sml.pm line 19.

Subroutine sml::evaluate_algorithm redefined at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/sml.pm line 19.

Subroutine sml::predict_classification redefined at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/sml.pm line 19.


A value of k=5 neighbors was used to make predictions. You may experiment with larger k
values to increase accuracy. Running the example prints the accuracy scores achieved on each
fold and the mean of those scores. We can see that the mean accuracy of 23% is better than the
baseline of 16%, but is quite poor in general. This is because of the large number of classes
making accuracy a poor judge of skill on this problem. This fact, combined with the fact that
many of the classes have few or one example also makes the problem challenging.

We can also model the dataset as a regression predictive modeling problem. This is because
the class values have a natural ordinal relationship as they are years.


# Abalone Case Study as Regression

Regression may be a more useful way to model this problem given the large number of classes
and sparseness of some class values. We can easily change our above example to regression by
changing KNN to predict the mean of the neighbors and using Root Mean Squared Error to
evaluate predictions. Below is a complete working example with these changes.

In [11]:

# Calculate root mean squared error
my $rmse_metric = sub{
    my ($self, $actual, $predicted) = @_;
    my $sum_error = 0.0;
    for my $i (0 .. @$actual - 1) {
        my $prediction_error = $predicted->[$i] - $actual->[$i];
        $sum_error += ($prediction_error ** 2);
    }
    my $mean_error = $sum_error / @$actual;
    return sqrt($mean_error);
};
sml->add_to_class('sml','rmse_metric',$rmse_metric);

# Evaluate an algorithm using a cross-validation split

my $evaluate_algorithm = sub{
    my ($self, $dataset, $algorithm, $n_folds, @args) = @_;
    my $folds = sml->cross_validation_split($dataset, $n_folds);
    my @scores;
    for my $fold (@$folds) {
        my @train_set = @$folds;
        @train_set = grep { $_ ne $fold } @train_set;
        @train_set = map @$_, @train_set;
        my @test_set;
        for my $row (@$fold) {
            my $row_copy = [@$row];
            push @test_set, $row_copy;
            $row_copy->[-1] = undef;
        }
        my $predicted = $algorithm->(\@train_set, \@test_set, @args);
        my @actual = map $_->[-1], @$fold;
        my $rmse = sml->rmse_metric(\@actual, $predicted);
        push @scores, $rmse;
    }
    return \@scores;
};
sml->add_to_class('sml','evaluate_algorithm',$evaluate_algorithm);

# Test the kNN on the Abalone dataset
srand(1);

# load and prepare data
my $filename = 'abalone.csv';
my $dataset = sml->load_csv($filename);
shift @$dataset;
for my $i (1 .. $#{$dataset->[0]}) {
    sml->str_column_to_float($dataset, $i);
}

# convert first column to integers
my $lookup = sml->str_column_to_int($dataset, 0);

# evaluate algorithm
my $n_folds = 5;
my $num_neighbors = 5;
my $scores = sml->evaluate_algorithm($dataset, \&k_nearest_neighbors, $n_folds, $num_neighbors);
print 'Scores: ' . join(', ', @$scores) . "\n";
printf 'Mean RMSE: %.3f\n', sum(@$scores) / @$scores;


Scores: 2.53596879408066, 2.76071673415212, 2.54045118346503, 2.78404349886865, 2.69085833199636
Mean RMSE: 2.662\n

1

Warning: Subroutine sml::rmse_metric redefined at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/sml.pm line 19.

Subroutine sml::evaluate_algorithm redefined at /usr/local/lib/perl5/site_perl/5.32.1/x86_64-linux/sml.pm line 19.


Running this example prints the RMSE on each fold and the mean RMSE across all folds.
We can see that the RMSE of 2.242 rings is better than the baseline of 3.222 rings. We also
have a model that is perhaps more useful in the domain with an performance that is easier to
understand.


 # 13.3 Extensions


This section lists extensions to the tutorial you may wish to consider to investigate.
* Tune KNN. Try larger and larger k values to see if you can improve the performance of
the algorithm on the Abalone dataset.
* Regression for Classification. Combine the approach used to make predictions for
regression problems (take the mean) with the classification approach to making predictions
(return an integer) and see if you can improve results.

* More Distance Measures. Implement other distance measures that you can use to find
similar historical data, such as Hamming distance, Manhattan distance and Minkowski
distance.
* Data Preparation. Distance measures are strongly affected by the scale of the input
data. Experiment with normalization and standardization data preparation methods in
order to improve results.
* More Problems. As always, experiment with the technique on more and different
classification and regression problems.


# 13.4 Review

In this tutorial you discovered how to implement the k-Nearest Neighbors algorithm from scratch
with Python. Specifically, you learned:
* How to implement k-Nearest Neighbors from scratch in Python.
* How to apply k-Nearest Neighbors to classification problems.
* How to apply k-Nearest Neighbors to regression problems.


## 13.4.1 Further Reading

* Section 3.5 Comparison of Linear Regression with K-Nearest Neighbors, page 104, An
Introduction to Statistical Learning, 2014.
http://amzn.to/2eeTyQX
* Section 18.8. Nonparametric Models, page 737, Artificial Intelligence: A Modern Approach,
2010.
http://amzn.to/2e3lFqP
* Section 7.4 K-Nearest Neighbors, page 159, and Section 13.5 K-Nearest Neighbors, page
350 Applied Predictive Modeling, 2013
http://amzn.to/2e3lNXF
* Section 4.7, Instance-based learning, page 128, Data Mining: Practical Machine Learning
Tools and Techniques, second edition, 2005.
http://amzn.to/2fj3SYY

## 13.4.2 Next

In the next tutorial, you will discover how to implement and apply the Learning Vector
Optimization algorithm for classification